1\. Write a function that converts numbers among the bin, dec, and hex representations (bin<->dec<->hex)

In [26]:
def convert(num, rep_from, rep_to):
    legal_values = ['bin', 'dec', 'hex']
    if rep_from not in legal_values or rep_to not in legal_values:
        raise ValueError('To representations values not legal')
    else:
        if rep_from == 'dec' and rep_to == 'bin':
            return bin(num)
        elif rep_from == 'bin' and rep_to == 'dec':
            return int(num, 2)
        elif rep_from == 'dec' and rep_to == 'hex':
            return hex(num)
        elif rep_from == 'hex' and rep_to == 'dec':
            return int(num, 16)
        elif rep_from == 'bin' and rep_to == 'hex':
            num_dec = int(num, 2)
            return hex(num_dec)
        elif rep_from == 'hex' and rep_to == 'bin':
            num_dec = int(num, 16)
            return  bin(num_dec)
        else:
            raise ValueError('To representations values not legal')
        

original_n = 5

# Dec to Bin
result1 = convert(original_n, 'dec', 'bin')
print(f'From Dec ({original_n}) to Bin ({result1})')

# Bin to Dec
result2 = convert(result1, 'bin', 'dec')
print(f'From Bin ({result1}) to Dec ({result2})')

# Dec to Hex
result3 = convert(original_n, 'dec', 'hex')
print(f'From Dec ({original_n}) to Hex ({result3})')

# Hex to Dec
result4 = convert(result3, 'hex', 'dec')
print(f'From Hex ({result3}) to Dec ({result4})')

# Bin to Hex
result5 = convert(result1, 'bin', 'hex')
print(f'From Bin ({result1}) to Hex ({result5})')

# Hex to Bin
result6 = convert(result3, 'hex', 'bin')
print(f'From Hex ({result3}) to Bin ({result6})')

From Dec (5) to Bin (0b101)
From Bin (0b101) to Dec (5)
From Dec (5) to Hex (0x5)
From Hex (0x5) to Dec (5)
From Bin (0b101) to Hex (0x5)
From Hex (0x5) to Bin (0b101)


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent) and check its validity.

In [30]:
# Original binary representation
bin_num = '00000011111000000000000000000000'

# Extract different part of the binary number
sign_str = bin_num[0]
exponent_str = bin_num[1:9]
mantissa_str = bin_num[9:]

# Calculates the sign
sign = 1 if sign_str == '0' else -1

# Calculates the exponent
exponent = int(exponent_str, 2)

# Calculates the mantissa
mantissa = 1
length = len(mantissa_str)
num = -1
for m in range(length):
    mantissa += int(mantissa_str[m]) * 2**(num)
    num -= 1
    
# Prints the results
print(f'Sign: {sign_str}, Exponent: {exponent_str}, Mantissa: {mantissa_str}')
print(f'Sign: {sign}, Exponent: {exponent}, Mantissa: {mantissa}')

# Calculates the float number and prints it
float_num = (-1)**sign * mantissa * 2**(exponent - 127)
print(f'Float number: {float_num}')

Sign: 0, Exponent: 00000111, Mantissa: 11000000000000000000000
Sign: 1, Exponent: 7, Mantissa: 1.75
Float number: -1.316553672920962e-36


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tip**: define two variables initialized to 1 and halve/double them for a sufficient amount of times to exceed the under/over-flow limits  

In [23]:
# NOTE: I'm not sure it was what you meant. Please provide a different 
# description if you want me to do something other than that.

num = 1
num_prev = None

while num != float('inf'):
    num_prev = num
    num = num * 2.0
    
print(f'Overflow limit: {num_prev}')

num = 1
num_prev = None

while num != 0:
    num_prev = num
    num = num / 2.0
    
print(f'Underflow limit: {num_prev}')

Overflow limit: 8.98846567431158e+307
Underflow limit: 5e-324


4\. Write a program to determine the machine precision

**Tip**: define a new variable by adding an increasingly smaller value (in the same way as the previous problem) and check when the addition starts to have no effect on the number

In [28]:
num = 0
increment = 100
loop = True

while loop:
    prev_num = num
    num += increment
    increment /= 2
    print(num)
    if prev_num == num:
        loop = False

100
150.0
175.0
187.5
193.75
196.875
198.4375
199.21875
199.609375
199.8046875
199.90234375
199.951171875
199.9755859375
199.98779296875
199.993896484375
199.9969482421875
199.99847412109375
199.99923706054688
199.99961853027344
199.99980926513672
199.99990463256836
199.99995231628418
199.9999761581421
199.99998807907104
199.99999403953552
199.99999701976776
199.99999850988388
199.99999925494194
199.99999962747097
199.99999981373549
199.99999990686774
199.99999995343387
199.99999997671694
199.99999998835847
199.99999999417923
199.99999999708962
199.9999999985448
199.9999999992724
199.9999999996362
199.9999999998181
199.99999999990905
199.99999999995453
199.99999999997726
199.99999999998863
199.99999999999432
199.99999999999716
199.99999999999858
199.9999999999993
199.99999999999966
199.99999999999983
199.99999999999991
199.99999999999997
200.0
200.0


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying the numerator and the denominator by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what has been previously obtained, and why?

(c) write a function that computes the roots of a quadratic equation accurately in all cases

In [55]:
from math import sqrt

def quadratic_eq(a, b, c):
    x1 = (-b + sqrt(b**2 - 4*a*c)) / (2*a)
    x2 = (-b - sqrt(b**2 - 4*a*c)) / (2*a)
    return (x1, x2)
    
def quadratic_eq_2(a, b, c):
    r = sqrt(b**2 - 4*a*c)
    x1 = 2*c / (-b - r)
    x2 = 2*c / (-b + r)
    return (x1, x2)

def quadratic_eq_stable(a, b, c):
    x1, _ = quadratic_eq_2(a, b, c)
    _, x2 = quadratic_eq(a, b, c)
    return (x1, x2)

print(quadratic_eq(0.001, 1000, 0.001))
print(quadratic_eq_2(0.001, 1000, 0.001))
print(quadratic_eq_stable(0.001, 1000, 0.001))

(-9.999894245993346e-07, -999999.999999)
(-1.000000000001e-06, -1000010.5755125057)
(-1.000000000001e-06, -999999.999999)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare it with the answer your program gives. The two will not agree perfectly. Why?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [38]:
def f(x):
    return x * (x - 1)

def derivative(x, delta=10**-2):
    return (f(x + delta) - f(x)) / delta

def derivative_analytically(x):
    return 2 * x - 1

# Because of algorithm stability and function conditioning the value of the 
# derivate calculated analytically is different from the other one
print(f'Derivative: {derivative(1)}')
print(f'Derivative (analytically): {derivative_analytically(1)}')

# Considering different values of delta we can observe that the best result has
# been obtained with delta=10^-8
print(f'Derivative (delta = 10*-4): {derivative(1, delta=10**-4)}')
print(f'Derivative (delta = 10*-6): {derivative(1, delta=10**-6)}')
print(f'Derivative (delta = 10*-8): {derivative(1, delta=10**-8)}')
print(f'Derivative (delta = 10*-10): {derivative(1, delta=10**-10)}')
print(f'Derivative (delta = 10*-12): {derivative(1, delta=10**-12)}')
print(f'Derivative (delta = 10*-14): {derivative(1, delta=10**-14)}')

Derivative: 1.010000000000001
Derivative (analytically): 1
Derivative (delta = 10*-4): 1.0000999999998899
Derivative (delta = 10*-6): 1.0000009999177333
Derivative (delta = 10*-8): 1.0000000039225287
Derivative (delta = 10*-10): 1.000000082840371
Derivative (delta = 10*-12): 1.0000889005833413
Derivative (delta = 10*-14): 0.9992007221626509


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which is known to be $I=\frac{\pi}{2}=1.57079632679...$.

Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a program to compute the integral with $N=100$. How does the result compare to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [36]:
from math import sqrt
import time

def f(x):
    return sqrt(1 - x**2)

def riemann_int(function, a, b, n):
    sumval = 0
    h = (b - a) / n
    for i in range(0, n - 1):
        current_x = a + i * h
        sumval = sumval + function(current_x) * h
    return sumval

# The approximate result is near the real one but the error is not negligible
print(riemann_int(f, -1, +1, 100))

clock = time.time()
riemann_int(f, -1, +1, 100)
time_el = time.time() - clock

print(f'Runs in 1 second: {1 / time_el}')
print(f'Runs in 60 seconds: {60 / time_el}')

1.565154305800824
Runs in 1 second: 14315.03071672355
Runs in 60 seconds: 858901.8430034129
